In [6]:
%config Completer.use_jedi = False

import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc
import os
from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing
from sedgen import geostatistics
from sedgen import visualization as viz
from sedgen.visualization import SedGenEvolution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2
wdir = r'D:/unif/masterthesis/jupyter/sedgenmasterthesisQ/figures/simulation/normal'
os.chdir(wdir)
minerals = ["Q", "P", "K", "B", "O", "A"]

modal_mineralogy = np.array([[0.16331256, 0.46387757, 0.12683457, 0.05758711, 0.01078412, 0.17760407],   # AZ
                             [0.27453254, 0.40124204, 0.24993859, 0.02509231, 0.00454988, 0.04464465],   # CA-EW
                             [0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603],   # CA-NS
                             [0.19781274, 0.41766034, 0.16121575, 0.06478527, 0.00894429, 0.14958161],   # GR
                             [0.25983144, 0.46392506, 0.15827298, 0.04370113, 0.00689158, 0.06737782],   # MT
                             [0.22419158, 0.48490082, 0.05641885, 0.11571367, 0.00189513, 0.11877509]])  # WA

csds_means = np.array([[0.120, 0.174, 0.112, 0.155, 0.066, 0.079],
                       [0.247, 0.333, 0.292, 0.212, 0.090, 0.127],
                       [0.309, 0.330, 0.244, 0.223, 0.120, 0.122],
                       [0.244, 0.269, 0.222, 0.251, 0.059, 0.107],
                       [0.220, 0.227, 0.160, 0.153, 0.078, 0.075],
                       [0.342, 0.385, 0.241, 0.481, 0.130, 0.130]])

csds_stds = np.array([[0.359, 0.356, 0.455, 0.399, 0.328, 0.452],
                      [0.986, 0.640, 0.497, 0.889, 0.275, 0.502],
                      [0.823, 0.683, 0.817, 0.819, 0.554, 0.782],
                      [0.312, 0.450, 0.444, 0.262, 0.378, 0.313],
                      [0.472, 0.415, 0.344, 0.363, 0.315, 0.599],
                      [0.542, 0.478, 0.386, 0.467, 0.450, 0.494]])



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%%time
for i in range(6):
    if i == 0:
        x = 0.697
    elif i == 1:
        x = 0.276
    elif i == 2:
        x = 0.2605
    elif i == 3:
        x = 0.314
    elif i == 4:
        x = 0.4315
    elif i == 5:
        x = 0.2775
        
    sedgen = SedGen(minerals, 1e8, modal_mineralogy[i], csds_means[i], csds_stds[i],
                    learning_rate = 1000,
                    auto_normalize_modal_mineralogy = True)
 
    sedgen_mech = \
        sedgen.weathering(
            operations=["intra_cb", "inter_cb"], 
            steps=100, inplace=False,
            display_mass_balance=False,
            pcg_model_stop=False)
    
    recommended_steps = searchplaceofsteps(x, sedgen_mech.pcg_additions, sedgen_mech.mcg_additions)
    print("recomended steps is:", recommended_steps)

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|
Counting interfaces...
Correcting interface arrays for consistency...
too much Q 1
all good P 0
all good K 0
all good B 0
all good O 0
too few A -1
Initializing crystal size array... |Q|P|K|B|O|A|
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...
Initializing discretization for model's weathering...
100/100
---SedGen model initialization finished succesfully---
recomended steps is: 11
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|
Counting interfaces...
Correcting interface arrays for consistency...
too much Q 9
too few P -2
too few K -2
too few B -1
too few O -3
too few A -1
Initi

In [3]:
def searchplaceofsteps(x, pcg, mcg):
    waardes = pcg/(mcg + pcg)
    minwaardes = waardes - x
    absolutevalues = abs(minwaardes)
    searched = absolutevalues.min()
    for i in range(len(waardes)):
        if absolutevalues[i] == searched:
            return(i)

In [18]:
sedgen_mech.pcg_additions
sedgen_mech.mcg_additions


array([     2,      4,      8,     16,     32,     63,    123,    228,
          419,    752,   1305,   2229,   3649,   5724,   8658,  12540,
        17710,  23939,  31634,  40469,  50500,  61310,  72768,  84778,
        96822, 108139, 118754, 127755, 135303, 140309, 143175, 143664,
       142123, 138377, 132183, 124534, 115247, 104904,  94152,  83264,
        72798,  62871,  53713,  45504,  38028,  31553,  25493,  20545,
        16237,  12638,   9744,   7488,   5674,   4261,   3145,   2231,
         1613,   1093,    766,    533,    343,    215,    137,     81,
           56,     37,     18,     15,     14,      8,      5,      3,
            2,      2,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=uint32)

array([              0,               0,               0,               0,
                     0,               1,               5,              28,
                    92,             264,             689,            1630,
                  3648,            7701,           15240,           28591,
                 50827,           87157,          144156,          232330,
                366126,          566667,          863253,         1296939,
               1926136,         2832915,         4130980,         5980297,
               8602460,        12306580,        17519410,        24831571,
              35054854,        49303477,        69099620,        96513011,
             134350428,       186399093,       257749755,       355216468,
             487880550,       667794448,       910884603,      1238107133,
            1676919386,      2263142805,      3043314316,      4077632744,
            5443644581,      7240820480,      9596218106,     12671453512,
           16671240540,  